In [57]:
import requests
import subprocess
import json
import pandas as pd
import folium
import time

In [3]:
gymlist = requests.get("https://api.toplogger.nu/v1/gyms")

In [9]:
gymdata = gymlist.json()

In [33]:
# The keys you are interested in
selected_keys = ['id', 'id_name', 'name', 'name_short', 'latitude', 'longitude', 'address', 'city', 'country', 'postal_code', 'nr_of_climbs', 'nr_of_boulders']
# Loop through the list and filter each dictionary
filtered_list = [{k: d[k] for k in selected_keys if k in d} for d in gymdata]
# Json dict to pandas DataFrame
df = pd.DataFrame(filtered_list)

In [41]:
df = df[df['country'] == 'NL']
df['gym_id'] = df['id']

In [43]:
df

,id,id_name,name,name_short,latitude,longitude,address,city,country,postal_code,nr_of_climbs,nr_of_boulders
0,1,grip_nijmegen_boulder,Grip Boulderhal,GRIP,51.8132651,5.8384377,Kamerlingh Onnestraat 35,Nijmegen,NL,6533 HK,173,173
1,2,arque_enschede,Arque,Arque,52.2072339,6.9000928,J.J. van Deinselaan 20,Enschede,NL,7541 BR,140,33
2,3,monk_rotterdam,Monk Rotterdam,Monk Rotterdam,51.9416071,4.4731773,Ceintuurbaan 183,Rotterdam,NL,3051 KC,180,180
3,4,grip_nijmegen_climbing,Stadsmuur Nijmegen,Stadsmuur Nijmegen,51.8218592,5.7879467,Van Rosenburgweg 52,Nijmegen,NL,6537 TM,207,0
4,5,monk_amsterdam,Monk Amsterdam,Monk Amsterdam,52.3834571,4.9291487,Aambeeldstraat 26,Amsterdam,NL,1021 KB,149,149
...,...,...,...,...,...,...,...,...,...,...,...,...
193,195,neoliet_boulder_tilburg,Boulder Tilburg,Boulder Tilburg,51.5657425,5.1111697,Jules verneweg 9,Tilburg,NL,5015 BD,132,132
194,196,kunststof_boulder_krachtstof,Krachtstof,Krachtstof,52.1584541,4.5120289,NaN,NaN,NL,NaN,177,177
197,199,monk_hilversum,Monk Hilversum,Monk Hilversum,52.2195681,5.1496425,Vreelandseweg 5,Hilversum,NL,1216 CG,189,189
206,208,boulderstation_enschede,Boulderstation,Boulderstation,52.2235127,6.8716011,Goolkatenweg 65,Enschede,NL,7521 AG,101,101


In [50]:
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')


<ipython-input-50-e46b6fb6b460>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
<ipython-input-50-e46b6fb6b460>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')


In [52]:
# Drop rows with NaN in 'latitude' or 'longitude'
df = df.dropna(subset=['latitude', 'longitude'])

In [53]:
# Create a map centered around the average latitude and longitude values
m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=7)

# Add circles to the map
for idx, row in df.iterrows():
    folium.Circle(
        location=[row['latitude'], row['longitude']],
        radius=row['nr_of_boulders']*10,  # Adjusting the radius size for visibility
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(m)

# Display the map
m.save('map.html')

In [78]:
base_url = "https://api.toplogger.nu/v1/gyms"
data = []

# Assuming df contains a column 'id'
for idx, row in df.head(3).iterrows():
    gym_id = row['id']
    url = f"{base_url}/{gym_id}/ranked_gym_users.json?climbs_type=boulders&ranking_type=grade&itemsPerPage=30"
    response = requests.get(url)
    if response.status_code == 200:  # If request was successful
        json_data = response.json()
        for user in json_data:
            user['gym_id'] = gym_id
            user['user_id'] = user.pop('id')  # Rename 'id' to 'user_id'
        data.extend(json_data)
    # Pause every 5 requests
    if (idx + 1) % 3 == 0:
        time.sleep(5)


In [79]:
small = data[0:100]

In [81]:
small

[{'uid': 1787952610,
  'full_name': 'Jesper Landman',
  'gender': 'male',
  'avatar': 'https://upload.toplogger.nu/avatars/191373509420550095718995506800505823.png',
  'score': '7.45003',
  'score_count': 10,
  'gym_id': 1,
  'user_id': 103520},
 {'uid': 8067844711,
  'full_name': 'Simon Boes',
  'gender': 'male',
  'avatar': 'https://upload.toplogger.nu/avatars/362717909653985271345032236313144719.png',
  'score': '7.40202',
  'score_count': 10,
  'gym_id': 1,
  'user_id': 6418},
 {'uid': 4334708070,
  'full_name': 'Minne de Man',
  'gender': 'male',
  'avatar': 'https://upload.toplogger.nu/avatars/158299505807503990675089925119935401.png',
  'score': '7.28401',
  'score_count': 10,
  'gym_id': 1,
  'user_id': 151583},
 {'uid': 8359093277,
  'full_name': 'Tom Hendriks ',
  'gender': 'male',
  'avatar': 'https://upload.toplogger.nu/avatars/899416879024237748230748626721645166.png',
  'score': '7.28368',
  'score_count': 10,
  'gym_id': 1,
  'user_id': 13025},
 {'uid': 2753414255,
  'fu

In [101]:
import requests
import time
import urllib.parse
import json

user_ascends = {}  # Initialize empty dictionary to store ascend data

base_url_ascends = "https://api.toplogger.nu/v1/ascends.json?json_params="

# Assuming you have previously populated 'data' from the previous loop

for entry in small:
    uid = entry['uid']
    gym_id = entry['gym_id']

    if time.time() - start_time >= 2:
        if is_sleeping:  
            is_sleeping = False
            start_time = time.time()
            continue  
        else:  
            is_sleeping = True
            time.sleep(2)
            continue
        
    # Create the URL using string formatting
    params = {
        "filters": {
            "used": True,
            "user": {"uid": str(uid)},
            "climb": {
                "gym_id": gym_id,
                "deleted": False,
                "live": True
            }
        }
    }
    json_params = urllib.parse.quote(json.dumps(params))
    url = f"{base_url_ascends}{json_params}&serialize_checks=true"
    
    response = requests.get(url)

    if response.status_code == 200:
        user_ascends_data = response.json()
        
        # Assuming the response data contains a list of ascends, we'll add each ascend to the dictionary with uid as the key
        user_ascends[uid] = user_ascends_data
    else:
        print(f"Failed to fetch data for UID: {uid} with status code {response.status_code}. Response: {response.text}")  # Debugging print

    # Pause every 2 seconds
    time.sleep(2)

# After the loop
print(user_ascends)


{4334708070: [{'id': 23911702, 'user_id': 151583, 'climb_id': 351120, 'topped': True, 'date_logged': '2023-09-05T15:01:45.000Z', 'used': True, 'checks': 2}, {'id': 23911703, 'user_id': 151583, 'climb_id': 351121, 'topped': True, 'date_logged': '2023-09-05T15:01:45.000Z', 'used': True, 'checks': 2}, {'id': 23911704, 'user_id': 151583, 'climb_id': 351122, 'topped': True, 'date_logged': '2023-09-05T15:01:45.000Z', 'used': True, 'checks': 2}, {'id': 23911705, 'user_id': 151583, 'climb_id': 351123, 'topped': True, 'date_logged': '2023-09-05T15:01:45.000Z', 'used': True, 'checks': 2}, {'id': 23911706, 'user_id': 151583, 'climb_id': 351124, 'topped': True, 'date_logged': '2023-09-05T15:01:45.000Z', 'used': True, 'checks': 2}, {'id': 23911707, 'user_id': 151583, 'climb_id': 351125, 'topped': True, 'date_logged': '2023-09-05T15:01:45.000Z', 'used': True, 'checks': 2}, {'id': 23911708, 'user_id': 151583, 'climb_id': 351126, 'topped': True, 'date_logged': '2023-09-05T15:01:45.000Z', 'used': True,

In [107]:
user_ascends

{4334708070: [{'id': 23911702,
   'user_id': 151583,
   'climb_id': 351120,
   'topped': True,
   'date_logged': '2023-09-05T15:01:45.000Z',
   'used': True,
   'checks': 2},
  {'id': 23911703,
   'user_id': 151583,
   'climb_id': 351121,
   'topped': True,
   'date_logged': '2023-09-05T15:01:45.000Z',
   'used': True,
   'checks': 2},
  {'id': 23911704,
   'user_id': 151583,
   'climb_id': 351122,
   'topped': True,
   'date_logged': '2023-09-05T15:01:45.000Z',
   'used': True,
   'checks': 2},
  {'id': 23911705,
   'user_id': 151583,
   'climb_id': 351123,
   'topped': True,
   'date_logged': '2023-09-05T15:01:45.000Z',
   'used': True,
   'checks': 2},
  {'id': 23911706,
   'user_id': 151583,
   'climb_id': 351124,
   'topped': True,
   'date_logged': '2023-09-05T15:01:45.000Z',
   'used': True,
   'checks': 2},
  {'id': 23911707,
   'user_id': 151583,
   'climb_id': 351125,
   'topped': True,
   'date_logged': '2023-09-05T15:01:45.000Z',
   'used': True,
   'checks': 2},
  {'id': 2

In [ ]:
### Calculate average rate of specific boulder
### Calculate difference between rating of climber of a boulder vs  average of all rating givers of this climb
### Calculate average difference per climber
### Regression: average difference per climber vs average grade (do better climbers sandbag more often?)